In [26]:
import pandas as pd
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_file
from functions import *

In [2]:
# load the data
df = GetData()
df.head()

,SourceFileId,TripEndTime,VehicleId,TripDistance,DriveFuel,DriveFuelEconomy,DistanceAvgLoadFactor,Drivetime,StopIdleTime,StopIdleFuel,...,AvgLoadFactor,HardBrakesPr1000miles,BrakesPr1000miles,TopGearMinusOneUsage,IdleFuelPercentage,EngineBrakePercentage,CruisePercentage,OverSpeedAPercentage,OverSpeedBPercentage,EngineFanPercentage
2,28424,8/19/2016,281,161448.1,28200.13,5.73,0.50,10727712,3976920,1034.50,...,0.45,0.371636,353.327168,0.056143,0.036684,0.111953,0.409951,0.013890,0.000617,0.073740
3,39301,2/5/2017,281,161448.1,28200.13,5.73,0.50,10727712,3976920,1034.50,...,0.45,0.371636,353.327168,0.056143,0.036684,0.111953,0.409951,0.013890,0.000617,0.073740
4,40235,5/23/2017,281,161448.1,28200.13,5.73,0.50,10727712,3976920,1034.50,...,0.45,0.371636,353.327168,0.056143,0.036684,0.111953,0.409951,0.013890,0.000617,0.073740
5,40616,9/13/2017,281,161448.3,28200.38,5.73,0.50,10727856,3976920,1034.50,...,0.45,0.371636,353.332925,0.056143,0.036684,0.111951,0.409945,0.013889,0.000617,0.073739
6,40694,11/6/2017,253,4572.5,774.00,5.91,0.51,286452,333288,34.25,...,0.45,0.656096,287.151449,0.303249,0.044251,0.476562,0.071635,0.255750,0.316451,1.188639


In [3]:
# get the adjusted DFE
df['dfe45'] = AdjustDFE(df)

In [4]:
km = KMeans(n_clusters=6)
X = df[['StopIdlePercent', 'DistanceAvgvehiclespeed','TopGearMinusOneUsage','HardBrakesPr1000miles', \
        'dfe45','CruisePercentage']]


In [5]:
clf = km.fit(X)

In [6]:
df['labels'] = clf.labels_

In [7]:
clusters = df[['VehicleId','labels']]

In [8]:
c = clusters.groupby(['VehicleId','labels']).size().reset_index()

In [9]:
c.rename(columns={0:'Counts'}, inplace=True)


In [10]:
t = c.groupby('VehicleId')['Counts'].sum()
t = t.to_dict()


In [11]:
c1 = c.sort_values('Counts', ascending=False).drop_duplicates('VehicleId')


In [12]:
c1['total'] = c1['VehicleId']

In [13]:
c1 = c1.replace({'total': t})

In [14]:
c1['ratio'] = c1['Counts']/c1['total']

In [15]:
c1

,VehicleId,labels,Counts,total,ratio
24,260,3,15,16,0.937500
62,280,3,14,16,0.875000
18,257,4,13,15,0.866667
38,272,0,11,13,0.846154
34,267,0,11,13,0.846154
65,283,2,11,14,0.785714
64,282,0,10,10,1.000000
30,264,0,9,15,0.600000
54,277,3,9,14,0.642857
27,262,0,8,15,0.533333


In [27]:
p = figure(title = 'Clusters')
colors = ["red", "olive", "darkred", "goldenrod", "skyblue", "orange", "salmon"]

In [45]:
x = df['DistanceAvgvehiclespeed']
y = df['dfe45']
c = df['labels'].values

In [46]:
clr = [colors[x] for x in c]

In [48]:
p.circle(x,y, color=clr)
show(p)